In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print("video data")
print(check_output(["ls", "./video_data"]).decode("utf8"))

print("frame data")
print(check_output(["ls", "./frame_data"]).decode("utf8"))
# Any results you write to the current directory are saved as output.

video data
traina0.tfrecord

frame data
traina0.tfrecord



In [2]:
import tensorflow as tf
import numpy as np
from IPython.display import YouTubeVideo


video_lvl_record = "./video_data/traina0.tfrecord"
frame_lvl_record = "./frame_data/traina0.tfrecord"

In [3]:
vid_ids = []
labels = []
mean_rgb = []
mean_audio = []

for example in tf.python_io.tf_record_iterator(video_lvl_record):
    tf_example = tf.train.Example.FromString(example)

    vid_ids.append(tf_example.features.feature['video_id'].bytes_list.value[0].decode(encoding='UTF-8'))
    labels.append(tf_example.features.feature['labels'].int64_list.value)
    mean_rgb.append(tf_example.features.feature['mean_rgb'].float_list.value)
    mean_audio.append(tf_example.features.feature['mean_audio'].float_list.value)

In [4]:
print('Number of videos in this tfrecord: ',len(mean_rgb))
print('First video feature length',len(mean_rgb[0]))
print('First 20 features of the first youtube video (',vid_ids[0],')')
print(mean_rgb[0][:20])

('Number of videos in this tfrecord: ', 1274)
('First video feature length', 1024)
('First 20 features of the first youtube video (', u'a0zzNorfSIw', ')')
[-0.6824246644973755, 0.35599029064178467, -0.5234265923500061, -0.12248212844133377, -0.014074370265007019, -0.311620831489563, -0.43062296509742737, 0.09638656675815582, 0.20725813508033752, -0.21594275534152985, 0.31706205010414124, 0.10410651564598083, 0.11305836588144302, -0.17438645660877228, -0.3506311774253845, 0.1398317962884903, -0.5600552558898926, 0.0593472495675087, 0.12069618701934814, -0.023518985137343407]


In [ ]:
# now, let's read the frame-level data
# due to execution time, we're only going to read the first video

feat_rgb = []
feat_audio = []

for example in tf.python_io.tf_record_iterator(frame_lvl_record):        
    tf_seq_example = tf.train.SequenceExample.FromString(example)
    print(tf_seq_example)
    n_frames = len(tf_seq_example.feature_lists.feature_list['audio'].feature)
    sess = tf.InteractiveSession()
    rgb_frame = []
    audio_frame = []
    # iterate through frames
    for i in range(n_frames):
        rgb_frame.append(tf.cast(tf.decode_raw(
                tf_seq_example.feature_lists.feature_list['rgb'].feature[i].bytes_list.value[0],tf.uint8)
                       ,tf.float32).eval())
        audio_frame.append(tf.cast(tf.decode_raw(
                tf_seq_example.feature_lists.feature_list['audio'].feature[i].bytes_list.value[0],tf.uint8)
                       ,tf.float32).eval())
        
        
    sess.close()
    feat_rgb.append(rgb_frame)
    feat_audio.append(audio_frame)
    break

In [ ]:
print('The first video has %d frames' %len(feat_rgb[0]))

In [ ]:
feat_rgb[0]

In [ ]:
first_video_feat = feat_rgb[0]

In [ ]:
pp = feat_rgb[0][:][0]
pp


In [ ]:
first_video_features_record = mean_rgb[0]
len(first_video_features_record)

In [ ]:
frame_level_record = feat_rgb[0]
a=np.mean(frame_level_record,axis = 0)

In [ ]:
len(a)

In [ ]:
a

In [ ]:
len(first_video_features_record)

In [ ]:
def Dequantize(feat_vector, max_quantized_value=2, min_quantized_value=-2):
  """Dequantize the feature from the byte format to the float format.
  Args:
    feat_vector: the input 1-d vector.
    max_quantized_value: the maximum of the quantized value.
    min_quantized_value: the minimum of the quantized value.
  Returns:
    A float vector which has the same shape as feat_vector.
  """
  assert max_quantized_value > min_quantized_value
  quantized_range = max_quantized_value - min_quantized_value
  scalar = quantized_range / 255.0
  bias = (quantized_range / 512.0) + min_quantized_value
  return feat_vector * scalar + bias

In [ ]:
test = Dequantize(a, max_quantized_value=2, min_quantized_value=-2)

In [ ]:
test

In [ ]:
len(frame_level_record)

In [ ]:
temp=[]
for item in range(len(frame_level_record)):
   temp.append(Dequantize(frame_level_record[item], max_quantized_value=2, min_quantized_value=-2))

In [ ]:
a=np.mean(temp,axis = 0)
a

Lets try to add labels to this data read

In [ ]:
# now, let's read the frame-level data
# due to execution time, we're only going to read the first video

feat_rgb = []
feat_audio = []
labels_frame=[]
for example in tf.python_io.tf_record_iterator(frame_lvl_record):        
    tf_seq_example = tf.train.SequenceExample.FromString(example)
    n_frames = len(tf_seq_example.feature_lists.feature_list['audio'].feature)
 

    labels_frame.append(tf_seq_example.context.feature['labels'].int64_list.value)


In [ ]:
labels_frame[0]

In [ ]:
labels[0]

Lets prototype the function that will receive num_videosx

In [15]:
import tensorflow as tf
import numpy as np
from IPython.display import YouTubeVideo
import random


video_lvl_record = "./video_data/traina0.tfrecord"
frame_lvl_record = "./frame_data/traina0.tfrecord"

# now, let's read the frame-level data
# due to execution time, we're only going to read the first video

feat_rgb = []
feat_audio = []
vid_ids = []
labels = []
mean_rgb = []
mean_audio = []


def Dequantize(feat_vector, max_quantized_value=2, min_quantized_value=-2):
  """Dequantize the feature from the byte format to the float format.
  Args:
    feat_vector: the input 1-d vector.
    max_quantized_value: the maximum of the quantized value.
    min_quantized_value: the minimum of the quantized value.
  Returns:
    A float vector which has the same shape as feat_vector.
  """
  assert max_quantized_value > min_quantized_value
  quantized_range = max_quantized_value - min_quantized_value
  scalar = quantized_range / 255.0
  bias = (quantized_range / 512.0) + min_quantized_value
  return feat_vector * scalar + bias

def frame_level_data_read(tf_seq_example):
  
    n_frames = len(tf_seq_example.feature_lists.feature_list['audio'].feature)

    rgb_frame = []
    audio_frame = []
    # iterate through frames
    for i in range(n_frames):
        rgb_frame.append(tf.cast(tf.decode_raw(
                tf_seq_example.feature_lists.feature_list['rgb'].feature[i].bytes_list.value[0],tf.uint8)
                       ,tf.float32).eval())
        audio_frame.append(tf.cast(tf.decode_raw(
                tf_seq_example.feature_lists.feature_list['audio'].feature[i].bytes_list.value[0],tf.uint8)
                       ,tf.float32).eval())
        
        
    return rgb_frame, audio_frame
  
  
def peturb_frames(rgb_frame,audio_frame, data_gen_param):
    frame_len= np.shape(rgb_frame)[0]
    rgb_test = random.sample(rgb_frame, int(frame_len*data_gen_param['keep_ratio'])) 
    audio_test = random.sample(audio_frame, int(frame_len*data_gen_param['keep_ratio'])) 
    temp=[]
    temp_audio=[]
    for item in range(len(rgb_test)):
        temp.append(Dequantize(rgb_test[item], max_quantized_value=2, min_quantized_value=-2))
        temp_audio.append(Dequantize(audio_test[item], max_quantized_value=2, min_quantized_value=-2))
    video_features=np.mean(temp,axis = 0)
    video_audio_features=np.mean(temp_audio,axis = 0)
    return video_features,video_audio_features


  

   
    
def generate_examples(example):
    tf_seq_example = tf.train.SequenceExample.FromString(example)
    vid_ids = []
    labels = []

    labels = tf_seq_example.context.feature['labels'].int64_list
    vid_ids_orig = tf_seq_example.context.feature['video_id'].bytes_list.value[0]
    example_list = []
   
    rgb_frame, audio_frame = frame_level_data_read(tf_seq_example)
    for i in range(data_gen_param['gen_videos']):
        
        if i<1:
          rgb_test, audio_test = peturb_frames(rgb_frame,audio_frame, {'keep_ratio':1, 'gen_videos':2})  
          vid_ids =vid_ids_orig
        else:
          vid_ids =vid_ids_orig+str(i) 
          rgb_test, audio_test = peturb_frames(rgb_frame,audio_frame, data_gen_param)
    
        example_list.append(tf.train.Example(features=tf.train.Features(feature={
           'labels':
                  tf.train.Feature(int64_list=labels),
           'video_id':
                  tf.train.Feature(bytes_list=tf.train.BytesList(value=vid_ids)),
           'mean_rgb':
                  tf.train.Feature(float_list=tf.train.FloatList(value=rgb_test.astype(float))),
           'mean_audio':
                  tf.train.Feature(float_list=tf.train.FloatList(value=audio_test.astype(float))),          
            })))
    return example_list
      
data_gen_param = {'keep_ratio':0.9, 'gen_videos':2}        
for example in tf.python_io.tf_record_iterator(frame_lvl_record):    
    sess = tf.InteractiveSession()
    test_example = generate_examples(example)
    sess.close()
    break

In [16]:
print(test_example[0])

features {
  feature {
    key: "labels"
    value {
      int64_list {
        value: 48
        value: 10
        value: 71
      }
    }
  }
  feature {
    key: "mean_audio"
    value {
      float_list {
        value: 1.15122640133
        value: 0.00916770845652
        value: 0.28018707037
        value: -0.00725770741701
        value: -0.555702567101
        value: 0.146237820387
        value: 1.29716718197
        value: 0.721948981285
        value: 0.284704089165
        value: -0.736463963985
        value: -0.139810815454
        value: -0.86474686861
        value: 0.580361664295
        value: 0.797669887543
        value: -0.64760273695
        value: 1.25823867321
        value: -0.586828649044
        value: -0.845611274242
        value: 0.148373156786
        value: -0.634051680565
        value: -1.0531462431
        value: 0.239862725139
        value: -0.165352329612
        value: -1.14874184132
        value: 0.346053034067
        value: 1.12946367264
     

In [ ]:
vid_ids = []
labels = []

labels.append(tf_seq_example.context.feature['labels'].int64_list)
vid_ids.append(tf_seq_example.context.feature['video_id'].bytes_list.value[0])

In [ ]:
type(labels[0])
type(vid_ids[0])

In [ ]:
print(vid_ids)

In [ ]:
  example = tf.train.Example(features=tf.train.Features(feature={
      'labels':
          tf.train.Feature(int64_list=labels[0]),
      'video_id':
          tf.train.Feature(bytes_list=tf.train.BytesList(value=vid_ids)),
      'mean_rgb':
          tf.train.Feature(float_list=tf.train.FloatList(value=rgb_test.astype(float))),
      'mean_audio':
          tf.train.Feature(float_list=tf.train.FloatList(value=rgb_test.astype(float))),          
    }))  

example

In [7]:
print(example)

features {
  feature {
    key: "labels"
    value {
      int64_list {
        value: 48
        value: 10
        value: 71
      }
    }
  }
  feature {
    key: "mean_audio"
    value {
      float_list {
        value: -0.682424485683
        value: 0.355990380049
        value: -0.523426353931
        value: -0.122481949627
        value: -0.0140742529184
        value: -0.311620771885
        value: -0.43062287569
        value: 0.0963866636157
        value: 0.20725813508
        value: -0.215942636132
        value: 0.317062228918
        value: 0.104106649756
        value: 0.11305847764
        value: -0.174386352301
        value: -0.350631207228
        value: 0.139831900597
        value: -0.560055136681
        value: 0.0593473538756
        value: 0.120696239173
        value: -0.0235188771039
        value: -0.373134046793
        value: -0.188265860081
        value: -0.0675390139222
        value: -0.294456124306
        value: 0.0869420617819
        value: 0.219248

In [ ]:
vid_ids = []
labels = []
mean_rgb = []
mean_audio = []

for example in tf.python_io.tf_record_iterator(video_lvl_record):
    tf_example = tf.train.Example.FromString(example)

    vid_ids.append(tf_example.features.feature['video_id'].bytes_list.value[0].decode(encoding='UTF-8'))
    labels.append(tf_example.features.feature['labels'].int64_list.value)
    mean_rgb.append(tf_example.features.feature['mean_rgb'].float_list.value)
    mean_audio.append(tf_example.features.feature['mean_audio'].float_list.value)

In [ ]:
print(tf_example)

In [ ]:
example = tf.train.Example(features=tf.train.Features(feature={
      'labels':
          tf.train.Feature(int64_list=labels[0]),
      'video_id':
          tf.train.Feature(bytes_list=tf.train.BytesList(value=[video_id])),
      'mean_rgb':
          tf.train.Feature(float_list=tf.train.FloatList(value=mean_rgb)),
      'mean_audio':
          tf.train.Feature(float_list=tf.train.FloatList(value=mean_audio)),          
    }))    